## LAB 5 (Not compolsory) ##  
### Connecting Spark to Snowflake managed Iceberg tables without Polaris Catalog ###  

In [2]:
import os

In the next cell you need to replace `<user_env>` with the directory of your anaconda environment. This allows the Jupyter notebook to find your spark environment.
It should look something like: `'/Users/mstarc/anaconda3/envs/iceberg-lab-techup/lib/python3.12/site-packages/pyspark'` when done.

In [ ]:
os.environ['SPARK_HOME'] = '<user_env>/lib/python3.12/site-packages/pyspark'

In [4]:
import findspark
findspark.init()
findspark.find()

'/Users/mstarc/anaconda3/envs/iceberg-lab-techup2/lib/python3.12/site-packages/pyspark'

# Environment Variables

In the next cell you need to replace `<private_key_location>` with the location of your private key. It should look something like: `/Users/mstarc/.ssh/rsa_key.p8` when done.  You will also need to replace `<username>` with your CAS2 username.

In [4]:
os.environ['SNOWFLAKE_CATALOG_URI'] = "jdbc:snowflake://sfpscogs-aws_cas2.snowflakecomputing.com"
os.environ['SNOWFLAKE_ROLE'] = "ACCOUNTADMIN"
os.environ['SNOWFLAKE_USERNAME'] = "<USERNAME>"
os.environ['SNOWFLAKE_PASSWORD'] = "<private_key_location>"

There is no need to change any of the values in the below cells. I have provided the AWS credentials to simplify the process.

In [6]:
os.environ['PACKAGES'] = "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.1,net.snowflake:snowflake-jdbc:3.14.2,software.amazon.awssdk:bundle:2.20.160,software.amazon.awssdk:url-connection-client:2.20.160"
os.environ['AWS_REGION'] = "us-west-2"
os.environ['AWS_ACCESS_KEY_ID'] = "***REMOVED***R"
os.environ['AWS_SECRET_ACCESS_KEY'] = "***REMOVED***"
# If you are using session credentials instead of simple name/secret credentials, also set the environment variable below
#os.environ['AWS_SESSION_TOKEN'] = "<your aws session token>"

# Run Spark 

In [7]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
# Create SparkSession, for AWS
spark = SparkSession.builder.appName('iceberg_lab')\
    .config('spark.jars.packages', os.environ['PACKAGES'])\
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/mstarc/anaconda3/envs/iceberg-lab-techup2/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mstarc/.ivy2/cache
The jars for the packages stored in: /Users/mstarc/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
net.snowflake#snowflake-jdbc added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f3937dce-aa33-4c9b-9a14-2ed99d78ee12;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.1 in central
	found net.snowflake#snowflake-jdbc;3.14.2 in central
	found software.amazon.awssdk#bundle;2.20.160 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.20.160 in central
	found software.amazon.awssdk#utils;2.20.160 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found software.amazon.awssdk#annotations;2.20.160 in central
	found org.slf4j#sl

In [9]:
# Create SparkSession, specifically for GCP where GCS needs the shaded jar as described here:
# https://github.com/GoogleCloudDataproc/hadoop-connectors/blob/master/gcs/INSTALL.md#troubleshooting-the-installation
spark = SparkSession.builder.appName('iceberg_lab')\
    .config('spark.jars', 'https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.18/gcs-connector-hadoop3-2.2.18-shaded.jar')\
    .config('spark.jars.packages', os.environ['PACKAGES'])\
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
    .getOrCreate()

24/10/03 15:37:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Spark configurations for all clouds
Regardless which cloud your Snowflake account is in, set the following configurations for Spark

In [10]:
spark.conf.set("spark.sql.defaultCatalog", "snowflake_catalog")
spark.conf.set("spark.sql.catalog.snowflake_catalog", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.snowflake_catalog.catalog-impl", "org.apache.iceberg.snowflake.SnowflakeCatalog")
spark.conf.set("spark.sql.catalog.snowflake_catalog.uri", os.environ['SNOWFLAKE_CATALOG_URI'])
spark.conf.set("spark.sql.catalog.snowflake_catalog.jdbc.role", os.environ['SNOWFLAKE_ROLE'])
spark.conf.set("spark.sql.catalog.snowflake_catalog.jdbc.user", os.environ['SNOWFLAKE_USERNAME'])
spark.conf.set("spark.sql.catalog.snowflake_catalog.jdbc.private_key_file", os.environ['SNOWFLAKE_PASSWORD'])
spark.conf.set("spark.sql.iceberg.vectorization.enabled", "false")

### Spark configurations for AWS
If your Snowflake account and object storage are on AWS, set these additional Spark configurations.

In [11]:
spark.conf.set("spark.sql.catalog.snowflake_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
# If you are using session credentials instead of simple name/secret credentials, use the credentials provider configuration below instead of the line above below
#spark.conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
spark.conf.set("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID'])
spark.conf.set("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY'])
# If you are using session credentials instead of simple name/secret credentials, also set the configuration below
#spark.conf.set("spark.hadoop.fs.s3a.session.token", os.environ['AWS_SESSION_TOKEN'])
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
spark.conf.set("spark.hadoop.fs.s3a.endpoint.region", os.environ['AWS_REGION'])

# Read Snowflake-managed Iceberg Tables #   
You will also need to replace `<username>` with your CAS2 username.

In [12]:
spark.sql("SHOW NAMESPACES IN <username>").show(20, False)

+-------------------------+
|namespace                |
+-------------------------+
|MSTARC.INFORMATION_SCHEMA|
|MSTARC.LAB3              |
|MSTARC.MSTARC            |
|MSTARC.PUBLIC            |
|MSTARC.STEP              |
|MSTARC.STEP2             |
|MSTARC.STEP3             |
+-------------------------+



Replace `<namespace>` with the value from the previous step relating to the namespace you created in LAB3. It should be like `MSTARC.LAB3`

In [13]:
spark.sql("USE <namespae>")

DataFrame[]

In [15]:
spark.sql("SHOW TABLES").show(20, False)

+-----------+------------------+-----------+
|namespace  |tableName         |isTemporary|
+-----------+------------------+-----------+
|MSTARC.LAB3|TEST_TABLE_MANAGED|false      |
+-----------+------------------+-----------+



Replace `<snowflake managed table>` with `namespace.tableName` from the previous step

In [19]:
df = spark.table("<snowflake managed table>")
df.show(20, False, True)

-RECORD 0---
 COL1 | 3   



Once you seeyour record from the Snowflake managed Iceberg table the lab is complete.